This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')



Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [43]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2014-01-01&end_date=2014-01-01&transform=rdiff"
params = dict(api_key=API_KEY)
res = requests.get(url, params=params)

In [44]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
test_json = res.json()
test_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': 'rdiff',
  'column_index': None,
  'start_date': '2014-01-01',
  'end_date': '2014-01-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

In [34]:
len(test_json)

1

In [35]:
test_json['dataset']['data']

[]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Collecting data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [149]:
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&transform=rdiff"
params = dict(api_key=API_KEY)
d = requests.get(url, params=params)

Parsing the data as JSON

In [150]:
d_json = d.json()

Finding the number of days of trading in this year:

In [151]:
len(d_json['dataset']['data'])

254

Making a nested list with all of the values for each of the following: 'Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume'

In [152]:
lst = [[] for j in range(7)]

for i in range(254):
    for n in range(7):
        lst[n].append(d_json['dataset']['data'][i][n])                    

Making a dictionary called AFX_X_dict with the keys being 'Date', 'Open', 'High', 'Low', 'Close', 'Change', and 'Traded Volume' and the values being the list of values from the nested list made in the cell above.

In [153]:
AFX_X_dict = {}
for i in range(7):
    AFX_X_dict[d_json['dataset']['column_names'][i]] = lst[i]

Does opening price contain any None values?

In [154]:
AFX_X_dict['Open'].count(None)

5

Yes, opening price contains None values.

Highest opening price

In [155]:
highest_opening_price = 0
for i in range(254):
    if AFX_X_dict['Open'][i] is not None:
        if AFX_X_dict['Open'][i] > highest_opening_price:
            highest_opening_price = AFX_X_dict['Open'][i]
highest_opening_price

0.041013824884793

Lowest opening price

In [156]:
lowest_opening_price = 1000000
for i in range(254):
    if AFX_X_dict['Open'][i] is not None:
        if AFX_X_dict['Open'][i] < lowest_opening_price:
            lowest_opening_price = AFX_X_dict['Open'][i]
lowest_opening_price

-0.084928229665072

Does highest price contain any None values?

In [157]:
AFX_X_dict['High'].count(None)

0

No values of highest price are None.

Does lowest price contain any None values?

In [158]:
AFX_X_dict['Low'].count(None)

0

No values of lowest price are None.

Largest change in any one day (based on High and Low price)

In [159]:
largest_change = 0
for i in range(254):
    change = AFX_X_dict['High'][i] - AFX_X_dict['Low'][i] 
    if change > largest_change:
        largest_change = change
largest_change

0.039492791920221

Are there any None values in closing prices?

In [160]:
AFX_X_dict['Close'].count(None)

0

No values of closing price are None

Largest change between any two days (based on Closing Price):
Is it just the largest change between any two **consecutive** days or the largest change overall?
I have done both, respectively, in the following two cells

In [161]:
largest_change = 0
for i in range(253):
    change = abs(AFX_X_dict['Close'][i] - AFX_X_dict['Close'][i + 1]) 
    if change > largest_change:
        largest_change = change
largest_change

0.056715761814445

In [162]:
AFX_X_dict['Close'].sort()
AFX_X_dict['Close'][-1] - AFX_X_dict['Close'][0]

0.097627197430696

Does Traded Volume contain any None values?

In [163]:
AFX_X_dict['Traded Volume'].count(None)

0

No values of Traded Volume are None.

Average daily Traded Volume:

In [164]:
sum = 0
for i in range(254):
    sum += AFX_X_dict['Traded Volume'][i]

In [165]:
ave = sum / 254
ave

7.33419175718327

Sorting the daily traded volume list in increasing order

In [166]:
AFX_X_dict['Traded Volume'].sort()

Since the number of days is even, determining the index of the two values needed to calculate median (index must be subtracted by 1 to account for 0-indexing).

In [167]:
middle_1_index = 254 // 2 - 1
middle_2_index = middle_1_index + 1

Getting the two values via their indices

In [168]:
middle_1 = AFX_X_dict['Traded Volume'][middle_1_index]
middle_2 = AFX_X_dict['Traded Volume'][middle_2_index]

Calculating median trading volume during this year:

In [169]:
median_trading_volume = (middle_1 + middle_2) / 2
median_trading_volume

-0.000971311428940335

**I do not understand how any traded volume values can be negative unless they take into account the price of the stock**